In [ ]:
!pip install torch torchvision matplotlib grad-cam


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda:0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy
from google.colab import drive
from PIL import Image


# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Data preparation
data_dir = '/content/drive/My Drive/final'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Custom Dataset class for test data without subfolders
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        self.image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path  # No label

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
    'val': datasets.ImageFolder(os.path.join(data_dir, 'validation'), data_transforms['val']),
    'test': TestDataset(os.path.join(data_dir, 'test'), data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=0),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=0),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=0)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

print(f'Classes: {class_names}')

# Define the CNN Model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

# Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Training Function with Debugging Outputs
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            phase_start_time = time.time()

            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                batch_start_time = time.time()

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                batch_end_time = time.time()
                print(f'Batch {i+1}/{len(dataloaders[phase])} - Time: {batch_end_time - batch_start_time:.4f}s')

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            print(f'{phase} Phase Time: {time.time() - phase_start_time:.4f}s')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

# Save the model
torch.save(model_ft.state_dict(), '/content/drive/My Drive/dance_model.pth')

Streaming output truncated to the last 5000 lines.
Batch 126/186 - Time: 0.1253s
Batch 127/186 - Time: 0.1249s
Batch 128/186 - Time: 0.1247s
Batch 129/186 - Time: 0.1246s
Batch 130/186 - Time: 0.1247s
Batch 131/186 - Time: 0.1248s
Batch 132/186 - Time: 0.1248s
Batch 133/186 - Time: 0.1248s
Batch 134/186 - Time: 0.1250s
Batch 135/186 - Time: 0.1231s
Batch 136/186 - Time: 0.1250s
Batch 137/186 - Time: 0.1249s
Batch 138/186 - Time: 0.1249s
Batch 139/186 - Time: 0.1255s
Batch 140/186 - Time: 0.1250s
Batch 141/186 - Time: 0.1257s
Batch 142/186 - Time: 0.1249s
Batch 143/186 - Time: 0.1249s
Batch 144/186 - Time: 0.1250s
Batch 145/186 - Time: 0.1249s
Batch 146/186 - Time: 0.1227s
Batch 147/186 - Time: 0.1246s
Batch 148/186 - Time: 0.1246s
Batch 149/186 - Time: 0.1249s
Batch 150/186 - Time: 0.1249s
Batch 151/186 - Time: 0.1246s
Batch 152/186 - Time: 0.1250s
Batch 153/186 - Time: 0.1229s
Batch 154/186 - Time: 0.1248s
Batch 155/186 - Time: 0.1248s
Batch 156/186 - Time: 0.1250s
Batch 157/186 - Tim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
model = '/content/drive/MyDrive/dance_model.pth'

In [5]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from PIL import Image
import os

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Data preparation
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom Dataset class for test data without subfolders
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        self.image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Mount Google Drive
drive.mount('/content/drive')

# Prepare test dataset and dataloader
test_dir = '/content/drive/My Drive/final/test'
test_dataset = TestDataset(test_dir, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Load the saved model
class_names = ['bharatanatyam', 'kathak', 'kathakali', 'kuchipudi', 'manipuri', 'mohiniyattam', 'odissi', 'sattriya']

model_ft = models.resnet18(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft.load_state_dict(torch.load('/content/drive/My Drive/dance_model.pth'))
model_ft = model_ft.to(device)
model_ft.eval()

# Define the testing function
def evaluate_model(model, dataloader, class_names):
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # Print predictions (or you could save them to a file)
    for idx, pred in enumerate(predictions):
        print(f'Image {idx + 1}: Predicted class = {class_names[pred]}')

# Evaluate the model
evaluate_model(model_ft, test_loader, class_names)


Using device: cuda:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Image 1: Predicted class = bharatanatyam
Image 2: Predicted class = bharatanatyam
Image 3: Predicted class = kathakali
Image 4: Predicted class = kathakali
Image 5: Predicted class = bharatanatyam
Image 6: Predicted class = kathakali
Image 7: Predicted class = kathakali
Image 8: Predicted class = bharatanatyam
Image 9: Predicted class = manipuri
Image 10: Predicted class = kathak
Image 11: Predicted class = kathakali
Image 12: Predicted class = kathakali
Image 13: Predicted class = kuchipudi
Image 14: Predicted class = kathakali
Image 15: Predicted class = kathak
Image 16: Predicted class = bharatanatyam
Image 17: Predicted class = kathak
Image 18: Predicted class = kathakali
Image 19: Predicted class = kuchipudi
Image 20: Predicted class = kathakali
Image 21: Predicted class = manipuri
Image 22: Predicted class = kathakali
Image 23: Pred